Notes:
- check at what point is it optimum to subset the data e.g., in time, with intake-esm

In [1]:
## Loading libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import xesmf as xe
import dask
import season_util as su

In [2]:
from ncar_jobqueue import NCARCluster
from distributed import Client
cluster = NCARCluster(project="UCGD0006", processes=2, cores=4)
cluster.adapt(minimum_jobs=1, maximum_jobs=10)
client = Client(cluster)
cluster

In [3]:
import intake
var= ["ua"]
col = intake.open_esm_datastore("/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cmip6.json")
#cat=col.search(activity_id="CMIP", source_id="CESM2",experiment_id="1pctCO2", variable_id=var,
# table_id="Amon", grid_label="gn")
cat=col.search(activity_id="CMIP",experiment_id="historical", variable_id=var,
table_id="Amon", grid_label="gn", time_range="185001-201412")

In [4]:
dset_dict = cat.to_dataset_dict(cdf_kwargs={'chunks': {"time": 36}, 
                                    'decode_times': True})

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 7 group(s)


In [5]:
dset_dict.keys()

dict_keys(['CMIP.BCC.BCC-ESM1.historical.Amon.gn', 'CMIP.CCCma.CanESM5.historical.Amon.gn', 'CMIP.MIROC.MIROC-ES2L.historical.Amon.gn', 'CMIP.NCAR.CESM2.historical.Amon.gn', 'CMIP.NCAR.CESM2-WACCM.historical.Amon.gn', 'CMIP.NUIST.NESM3.historical.Amon.gn', 'CMIP.UA.MCM-UA-1-0.historical.Amon.gn'])

In [6]:
dset_dict['CMIP.UA.MCM-UA-1-0.historical.Amon.gn']=dset_dict['CMIP.UA.MCM-UA-1-0.historical.Amon.gn'].rename({'latitude': 'lat', 'longitude': 'lon'})

In [33]:
#ds_mean = {}
#for key, ds in dset_dict.items():
#    dsua=ds.ua
#    #print(dsua)
#    try:
#        dsua = dsua.sel(time=slice('1979-01-01','2006-01-01'), plev=70000.).mean(dim=["member_id"])
#        ds_mean[key] = dsua.mean(dim=["lon"])
#    except:
#        dsua=dsua.rename({'latitude': 'lat', 'longitude': 'lon'})
#        #print(dsua)
#        #ds_mean[key] = dsua.sel(time=slice('1979-01-01','2006-01-01'), plev=70000.).mean(dim=["member_id"])
#        #ds_mean[key] = dsua.mean(dim=["lon"])
#        
#     #   print(key)
#ds_mean

<xarray.DataArray 'ua' (time: 324, lat: 80, lon: 96)>
dask.array<mean_agg-aggregate, shape=(324, 80, 96), dtype=float32, chunksize=(36, 80, 96), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -88.29 -86.07 -83.84 -81.61 ... 83.84 86.07 88.29
    plev     float64 7e+04
  * lon      (lon) float64 0.0 3.75 7.5 11.25 15.0 ... 345.0 348.8 352.5 356.2
  * time     (time) object 1979-01-17 00:00:00 ... 2005-12-17 00:00:00


{'CMIP.BCC.BCC-ESM1.historical.Amon.gn': <xarray.DataArray 'ua' (time: 324, lat: 64)>
 dask.array<mean_agg-aggregate, shape=(324, 64), dtype=float32, chunksize=(36, 64), chunktype=numpy.ndarray>
 Coordinates:
     plev     float64 7e+04
   * lat      (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 79.53 82.31 85.1 87.86
   * time     (time) object 1979-01-16 12:00:00 ... 2005-12-16 12:00:00,
 'CMIP.CCCma.CanESM5.historical.Amon.gn': <xarray.DataArray 'ua' (time: 324, lat: 64)>
 dask.array<mean_agg-aggregate, shape=(324, 64), dtype=float32, chunksize=(36, 64), chunktype=numpy.ndarray>
 Coordinates:
     plev     float64 7e+04
   * lat      (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 79.53 82.31 85.1 87.86
   * time     (time) object 1979-01-16 12:00:00 ... 2005-12-16 12:00:00,
 'CMIP.MIROC.MIROC-ES2L.historical.Amon.gn': <xarray.DataArray 'ua' (time: 324, lat: 64)>
 dask.array<mean_agg-aggregate, shape=(324, 64), dtype=float32, chunksize=(36, 64), chunktype=numpy.ndarray>
 Coordinates:
  

In [15]:
ds_mean = {}
for key, ds in dset_dict.items():
    ds_mean[key] = ds.sel(plev=[70000.]).mean(dim=["member_id", "lon"])
    
ds_mean


{'CMIP.BCC.BCC-ESM1.historical.Amon.gn': <xarray.Dataset>
 Dimensions:   (bnds: 2, lat: 64, plev: 1, time: 1980)
 Coordinates:
   * lat       (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
   * plev      (plev) float64 7e+04
   * time      (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
 Dimensions without coordinates: bnds
 Data variables:
     lon_bnds  (bnds) float64 dask.array<chunksize=(2,), meta=np.ndarray>
     lat_bnds  (lat, bnds) float64 dask.array<chunksize=(64, 2), meta=np.ndarray>
     ua        (time, plev, lat) float32 dask.array<chunksize=(36, 1, 64), meta=np.ndarray>,
 'CMIP.CCCma.CanESM5.historical.Amon.gn': <xarray.Dataset>
 Dimensions:   (bnds: 2, lat: 64, plev: 1, time: 1980)
 Coordinates:
   * lat       (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
   * plev      (plev) float64 7e+04
   * time      (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
 Dimensions without coordinates: bnds
 Data variables:
     lon_bnds 

In [16]:
import dask
ds_mean = dask.compute(ds_mean)
ds_mean

({'CMIP.BCC.BCC-ESM1.historical.Amon.gn': <xarray.Dataset>
  Dimensions:   (bnds: 2, lat: 64, plev: 1, time: 1980)
  Coordinates:
    * lat       (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
    * plev      (plev) float64 7e+04
    * time      (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  Dimensions without coordinates: bnds
  Data variables:
      lon_bnds  (bnds) float64 177.2 180.0
      lat_bnds  (lat, bnds) float64 -90.0 -86.48 -86.48 -83.7 ... 86.48 86.48 90.0
      ua        (time, plev, lat) float32 -1.35394 -1.1002313 ... -0.007226696,
  'CMIP.CCCma.CanESM5.historical.Amon.gn': <xarray.Dataset>
  Dimensions:   (bnds: 2, lat: 64, plev: 1, time: 1980)
  Coordinates:
    * lat       (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
    * plev      (plev) float64 7e+04
    * time      (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  Dimensions without coordinates: bnds
  Data variables:
      lon_bnds  (bnds) float64 177.2 180.

In [21]:
?su.season_mean
jja_mean = {}
for key, x in dset_dict.items():
     jja_mean[key]=su.season_mean(ds_mean[0][key],"ua",season="JJA")

/ncar/usr/jupyterhub/envs/cmip6-201910/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/ncar/usr/jupyterhub/envs/cmip6-201910/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


Signature: su.season_mean(ds, var, season='all', cal='none')
Docstring:
calculate climatological mean by season
Args: ds (xarray.Dataset): dataset
      var (str): variable to use
      season (str): "all", 'DJF', "MAM", "JJA", "SON"
      cal (str): "none"(default) or calendar used for weighting months by number of days
File:      ~/cmip6hackathon/season_util.py
Type:      function


In [26]:
print(jja_mean["CMIP.BCC.BCC-ESM1.historical.Amon.gn"].shape)

(1, 64)
